In [1]:
import dotenv
import dspy
from agentic_system.tools.search import search_web, search_pubmed_abstracts

In [ ]:
from contextlib import asynccontextmanager
from mcp import ClientSession, stdio_client, StdioServerParameters


@asynccontextmanager
async def load_mcp_tools(server_file_paths):
    server_connections = []
    all_dspy_tools = []
    try:
        for server_path in server_file_paths:
            params = StdioServerParameters(command="python", args=[server_path])
            client = stdio_client(params)
            read, write = await client.__aenter__()
            session = ClientSession(read, write)
            await session.__aenter__()
            await session.initialize()
            tools = await session.list_tools()
            dspy_tools = [
                dspy.Tool.from_mcp_tool(session, tool) for tool in tools.tools
            ]
            all_dspy_tools.extend(dspy_tools)
            server_connections.append((client, session))
        yield all_dspy_tools
    finally:
        for client, session in reversed(server_connections):
            await session.__aexit__(None, None, None)
            await client.__aexit__(None, None, None)

In [ ]:
# Set API key
dotenv.load_dotenv("../.env")


# Configure DSPy
class CompoundSummarizer(dspy.Signature):
    """Determine remaining cell count after applying a compound in a screening assay. Strictly base result on tool use and corresponding data."""

    compound_name: str = dspy.InputField(
        desc="Name of the compound to esimate toxicity for."
    )
    remaining_cells: int = dspy.OutputField(
        desc="""
        Esimate the number of cells left after the compound is applied in a screening assay.
        In this screen a 10 uM solution of the compound is suspending in DMSO and applied to well with 4000 primary human ventricular fibroblasts.
        """
    )
    confidence: float = dspy.OutputField(
        desc="""
        Confidence as probability (0-1) that predicted remaining cell count is accurate.
        Based this on the availability, quality, and relevance of the data used to make the prediction.
        """
    )


# dspy.configure(lm=dspy.LM("gemini/gemini-2.5-flash-preview-05-20", temperature=0.2))
dspy.configure(lm=dspy.LM("openai/o3-2025-04-16", temperature=1.0, max_tokens=25000))
dspy.configure_cache(
    enable_disk_cache=False,
    enable_memory_cache=False,
)

mcp_server_paths = [
    "../agentic_system/tools/chembl_mcp_server.py",
    "../agentic_system/tools/pubchem_mcp_server.py",
]


async def get_agent_results(compound_name: str):
    async with load_mcp_tools(mcp_server_paths) as dspy_tools:
        dspy_tools.extend([search_web, search_pubmed_abstracts])
        agent = dspy.ReAct(CompoundSummarizer, tools=dspy_tools, max_iters=4)
        result = await agent.aforward(compound_name=compound_name)
        return result


# Run
result = await get_agent_results("SNS-032")
result

Prediction(
    trajectory={'thought_0': 'First, I need to obtain the PubChem CID for SNS-032 so that I can look up its bioassay data and any cell-based screening results. I’ll start with a PubChem compound search.', 'tool_name_0': 'PUBCHEM__search_compounds', 'tool_args_0': {'query': 'SNS-032', 'search_type': 'name', 'max_records': 5}, 'observation_0': 'Execution error in PUBCHEM__search_compounds: \nTraceback (most recent call last):\n  File "/Users/roshankern/Desktop/Github/cf-compound-selection/backend/.venv/lib/python3.12/site-packages/dspy/predict/react.py", line 114, in aforward\n    trajectory[f"observation_{idx}"] = await self.tools[pred.next_tool_name].acall(**pred.next_tool_args)\n                                       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^\n  File "/Users/roshankern/Desktop/Github/cf-compound-selection/backend/.venv/lib/python3.12/site-packages/dspy/utils/callback.py", line 296, in async_wrapper\n    return await fn(instance, *ar

In [4]:
dspy.inspect_history(n=10)





[2025-06-11T17:19:03.099446]

System message:

Your input fields are:
1. `compound_name` (str): Name of the compound to esimate toxicity for.
2. `trajectory` (str):
Your output fields are:
1. `next_thought` (str): 
2. `next_tool_name` (Literal['CHEMBL__search_compounds', 'CHEMBL__get_compound_bioactivities', 'CHEMBL__get_activity_info', 'CHEMBL__get_assay_info', 'CHEMBL__get_mechanisms_of_action', 'CHEMBL__get_molecule_info', 'CHEMBL__get_drug_info', 'CHEMBL__get_drug_indications', 'CHEMBL__get_drug_warning', 'CHEMBL__search_targets', 'CHEMBL__get_target_information', 'CHEMBL__get_active_compounds', 'PUBCHEM__search_compounds', 'PUBCHEM__get_compound_info', 'PUBCHEM__search_by_smiles', 'PUBCHEM__search_by_inchi', 'PUBCHEM__search_by_cas_number', 'PUBCHEM__get_compound_synonyms', 'PUBCHEM__search_similar_compounds', 'PUBCHEM__substructure_search', 'PUBCHEM__superstructure_search', 'PUBCHEM__get_3d_conformers', 'PUBCHEM__analyze_stereochemistry', 'PUBCHEM__get_compound_properties', '

In [5]:
result.trajectory

{'thought_0': 'First, I need to obtain the PubChem CID for SNS-032 so that I can look up its bioassay data and any cell-based screening results. I’ll start with a PubChem compound search.',
 'tool_name_0': 'PUBCHEM__search_compounds',
 'tool_args_0': {'query': 'SNS-032', 'search_type': 'name', 'max_records': 5},
 'observation_0': 'Execution error in PUBCHEM__search_compounds: \nTraceback (most recent call last):\n  File "/Users/roshankern/Desktop/Github/cf-compound-selection/backend/.venv/lib/python3.12/site-packages/dspy/predict/react.py", line 114, in aforward\n    trajectory[f"observation_{idx}"] = await self.tools[pred.next_tool_name].acall(**pred.next_tool_args)\n                                       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^\n  File "/Users/roshankern/Desktop/Github/cf-compound-selection/backend/.venv/lib/python3.12/site-packages/dspy/utils/callback.py", line 296, in async_wrapper\n    return await fn(instance, *args, **kwargs)\n         